In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import random
import json
import os
import time


import tensorflow as tf

import tensorflow.contrib.legacy_seq2seq as seq2seq

from sklearn.model_selection import train_test_split

In [2]:
xo = np.load("iseq_n.npy")
yo = np.load("oseq_n.npy")

In [3]:
x = xo[:,:26,:]
y = yo[:,:26]

In [5]:
nxchars = x.shape[2]
print(nxchars)
ltokens = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '-', '+', '=', '#leq', '#neq', '#geq', '#alpha',
                            '#beta', '#lambda', '#lt', '#gt', 'x', 'y', '^', '#frac', '{', '}' ,' ']
print("#ltokens: ", len(ltokens))

30
#ltokens:  28


In [6]:
x_seq_length = len(x[0])
y_seq_length = len(y[0])- 1
print(x_seq_length, y_seq_length)

26 25


In [7]:
def batch_data(x, y, batch_size):
    shuffle = np.random.permutation(len(x))
    start = 0
#     from IPython.core.debugger import Tracer; Tracer()()
    x = x[shuffle]
    y = y[shuffle]
    while start + batch_size <= len(x):
        yield x[start:start+batch_size], y[start:start+batch_size]
        start += batch_size

In [8]:
batch_size = 512
nodes = 256
embed_size = 20

# Tensor where we will feed the data into graph
inputs = tf.placeholder(tf.float32, (None, x_seq_length, nxchars), 'inputs')
outputs = tf.placeholder(tf.int32, (None, None), 'output')
targets = tf.placeholder(tf.int32, (None, None), 'targets')

# Embedding layers
# input_embedding = tf.Variable(tf.random_uniform((len(char2numX), embed_size), -1.0, 1.0), name='enc_embedding')
output_embedding = tf.Variable(tf.random_uniform((len(ltokens)+1, embed_size), -1.0, 1.0), name='dec_embedding')
# date_input_embed = tf.nn.embedding_lookup(input_embedding, inputs)
date_output_embed = tf.nn.embedding_lookup(output_embedding, outputs)

with tf.variable_scope("encoding") as encoding_scope:
    lstm_enc = tf.contrib.rnn.BasicLSTMCell(nodes)
    _, last_state = tf.nn.dynamic_rnn(lstm_enc, inputs=inputs, dtype=tf.float32)

with tf.variable_scope("decoding") as decoding_scope:
    lstm_dec = tf.contrib.rnn.BasicLSTMCell(nodes)
    dec_outputs, _ = tf.nn.dynamic_rnn(lstm_dec, inputs=date_output_embed, initial_state=last_state)
#connect outputs to 
logits = tf.contrib.layers.fully_connected(dec_outputs, num_outputs=len(ltokens)+1, activation_fn=None) 
with tf.name_scope("optimization"):
    # Loss function
    loss = tf.contrib.seq2seq.sequence_loss(logits, targets, tf.ones([batch_size, y_seq_length]))
    # Optimizer
    optimizer = tf.train.RMSPropOptimizer(1e-3).minimize(loss)

Train the graph above:

In [9]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
print(y_train[0])

[28  2  8  3 18 23  2 11  3  0  7 27 13 27  6  6  8 27 27 27 27 27 27 27
 27 27]


In [10]:
def save(sess):
    saver = tf.train.Saver(None)

    save_path = saver.save(sess, save_path="seq_mod_128/model", global_step=None)
    print('model saved at %s' % save_path)

In [12]:
with tf.Session() as sess: 
    sess.run(tf.global_variables_initializer())
    restore(sess)
    epochs = 100
    for epoch_i in range(epochs):
        start_time = time.time()
        for batch_i, (source_batch, target_batch) in enumerate(batch_data(X_train, y_train, batch_size)):
            _, batch_loss, batch_logits = sess.run([optimizer, loss, logits],
                feed_dict = {inputs: source_batch,
                 outputs: target_batch[:, :-1],
                 targets: target_batch[:, 1:]})

        accuracy = np.mean(batch_logits.argmax(axis=-1) == target_batch[:,1:])
        print('Epoch {:3} Loss: {:>6.3f} Accuracy: {:>6.4f} Epoch duration: {:>6.3f}s'.format(epoch_i, batch_loss, 
                                                                          accuracy, time.time() - start_time))

        source_batch, target_batch = next(batch_data(X_test, y_test, batch_size))

        dec_input = np.zeros((len(source_batch), 1)) + len(ltokens)
        for i in range(y_seq_length):
            batch_logits = sess.run(logits,
                        feed_dict = {inputs: source_batch,
                         outputs: dec_input})
            prediction = batch_logits[:,-1].argmax(axis=-1)
            dec_input = np.hstack([dec_input, prediction[:,None]])

        print('Accuracy on test set is: {:>6.3f}'.format(np.mean(dec_input == target_batch)))     
        if epoch_i % 5 == 0:
            save(sess)
    
    save(sess)

INFO:tensorflow:Restoring parameters from seq_mod_128/model
model restored from seq_mod_128/model
Epoch   0 Loss:  0.150 Accuracy: 0.9539 Epoch duration: 39.531s
Accuracy on test set is:  0.906
model saved at seq_mod_128/model
Epoch   1 Loss:  0.144 Accuracy: 0.9568 Epoch duration: 39.299s
Accuracy on test set is:  0.879
Epoch   2 Loss:  0.137 Accuracy: 0.9602 Epoch duration: 39.267s
Accuracy on test set is:  0.873
Epoch   3 Loss:  0.128 Accuracy: 0.9646 Epoch duration: 39.297s
Accuracy on test set is:  0.913
Epoch   4 Loss:  0.120 Accuracy: 0.9627 Epoch duration: 39.186s
Accuracy on test set is:  0.904
Epoch   5 Loss:  0.142 Accuracy: 0.9598 Epoch duration: 39.277s
Accuracy on test set is:  0.893
model saved at seq_mod_128/model
Epoch   6 Loss:  0.114 Accuracy: 0.9668 Epoch duration: 39.230s
Accuracy on test set is:  0.905
Epoch   7 Loss:  0.111 Accuracy: 0.9677 Epoch duration: 39.338s
Accuracy on test set is:  0.905
Epoch   8 Loss:  0.128 Accuracy: 0.9623 Epoch duration: 39.274s
Accu

Epoch  79 Loss:  0.004 Accuracy: 0.9995 Epoch duration: 39.405s
Accuracy on test set is:  0.952
Epoch  80 Loss:  0.002 Accuracy: 0.9998 Epoch duration: 39.423s
Accuracy on test set is:  0.952
model saved at seq_mod_128/model
Epoch  81 Loss:  0.003 Accuracy: 0.9996 Epoch duration: 39.439s
Accuracy on test set is:  0.950
Epoch  82 Loss:  0.004 Accuracy: 0.9990 Epoch duration: 39.362s
Accuracy on test set is:  0.941
Epoch  83 Loss:  0.002 Accuracy: 0.9999 Epoch duration: 39.368s
Accuracy on test set is:  0.949
Epoch  84 Loss:  0.001 Accuracy: 1.0000 Epoch duration: 39.430s
Accuracy on test set is:  0.950
Epoch  85 Loss:  0.002 Accuracy: 0.9998 Epoch duration: 39.414s
Accuracy on test set is:  0.944
model saved at seq_mod_128/model
Epoch  86 Loss:  0.002 Accuracy: 1.0000 Epoch duration: 39.368s
Accuracy on test set is:  0.965
Epoch  87 Loss:  0.003 Accuracy: 0.9991 Epoch duration: 39.418s
Accuracy on test set is:  0.950
Epoch  88 Loss:  0.002 Accuracy: 0.9996 Epoch duration: 39.469s
Accura

Translate on test set

In [11]:
def restore(sess):
    saver = tf.train.Saver(None)
    path = "seq_mod_128/model"
    saver.restore(sess, save_path=path)
    print('model restored from %s' % path)

In [13]:
with tf.Session() as sess: 
    restore(sess)
    batch_size = 512
    source_batch, target_batch = next(batch_data(X_test, y_test, batch_size))

    dec_input = np.zeros((len(source_batch), 1)) + len(ltokens)
    for i in range(y_seq_length):
        batch_logits = sess.run(logits,
                    feed_dict = {inputs: source_batch,
                     outputs: dec_input})
        prediction = batch_logits[:,-1].argmax(axis=-1)
        dec_input = np.hstack([dec_input, prediction[:,None]])

    print('Accuracy on test set is: {:>6.3f}'.format(np.mean(dec_input == target_batch)))

INFO:tensorflow:Restoring parameters from seq_mod_128/model
model restored from seq_mod_128/model
Accuracy on test set is:  0.950


Let's randomly take two from this test set and see what it spits out:

In [ ]:
i = 102
print(dec_input[i,:])
print(ltokens)
seq = ""
for c in dec_input[i,1:]:
    c = int(c)
    if c != 28:
        seq += ltokens[c] 
        
print(seq)

seq = ""
for c in target_batch[i,1:]:
    c = int(c)
    
    if c != 28:
        seq += ltokens[c] 

print("Correct: ", seq)

In [14]:
with tf.Session() as sess: 
    restore(sess)
    batch_size = 512
    source_batch, target_batch = next(batch_data(X_test, y_test, batch_size))

    dec_input = np.zeros((len(source_batch), 1)) + len(ltokens)
    for i in range(y_seq_length):
        batch_logits = sess.run(logits,
                    feed_dict = {inputs: source_batch,
                     outputs: dec_input})
        prediction = batch_logits[:,-1].argmax(axis=-1)
        dec_input = np.hstack([dec_input, prediction[:,None]])

    print('Accuracy on test set is: {:>6.3f}'.format(np.mean(dec_input == target_batch)))
    cc = 0
    fcc = 0
    fc = 0
    fl = 0
    nl = 0
    for i in range(len(dec_input)):
        feq = False
        pseq = ""
        for c in dec_input[i,1:]:
            c = int(c)
            if c != 28:
                pseq += ltokens[c] 

        cseq = ""
        cseql = ""
        for c in target_batch[i,1:]:
            c = int(c)

            if c != 28:
                cseq += ltokens[c] 
                cseql += ltokens[c][0]
                if ltokens[c] == "#frac":
                    fc += 1
                    feq = True

        if pseq == cseq:
            cc += 1
            if feq:
                fcc += 1
        else:
            print("pseq: ", pseq)
            print("cseq: ", cseq)
        cseql = cseql.rstrip()
        if feq:
            fl += len(cseql)
        else:
            nl += len(cseql)
        
    print("Accuracy %.2f %%" % (cc/len(dec_input)*100))  
    print("Accuracy for fraction equations %.2f %%" % (fcc/fc*100))
    print("Accuracy for simple equations %.2f %%" % ((cc-fcc)/(len(dec_input)-fc)*100))
    print("Average length frac: %.1f" % (fl/fc))
    print("Average length simple: %.1f" % (nl/(len(dec_input)-fc)))

INFO:tensorflow:Restoring parameters from seq_mod_128/model
model restored from seq_mod_128/model
Accuracy on test set is:  0.953
pseq:  116+861 = -779           
cseq:  116+861 = 977            
pseq:  509y^4+587 #geq 1108        
cseq:  509y^4+587 #geq 1018        
pseq:  #frac{227}{385}#lambda^2-289 #leq 77  
cseq:  #frac{327}{385}#lambda^2-289 #leq 77  
pseq:  712#beta^3-732 #lt 16          
cseq:  712#beta^3-732 #lt -16         
pseq:  9x^3+697 #neq 160           
cseq:  9x^3+691 #neq 760           
pseq:  #frac{246}{770}#alpha^4+626 #geq 824 
cseq:  #frac{240}{770}#alpha^4+626 #geq 824 
pseq:  #frac{181}{231}-266 = -85    
cseq:  #frac{181}{451}-266 = -85    
pseq:  818#lambda^4+171 #leq 1106        
cseq:  818#lambda^4+171 #leq 1016        
pseq:  #frac{358}{103}+947 = 1297   
cseq:  #frac{350}{10}+947 = 1297    
pseq:  40#alpha^4-334 #geq -281         
cseq:  40#alpha^4-334 #leq -281         
pseq:  624x^4-514 #lt 81          
cseq:  624x^4-544 #lt 81          
pseq:  #frac{81}